In [11]:
!pip install --upgrade wandb -q
!pip install pytorch-msssim -q

#### Imports

In [12]:
from kaggle_secrets import UserSecretsClient

import numpy as np
import os
import wandb
from PIL import Image
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch
from torch.utils.data import DataLoader, Dataset

#### Setups

In [13]:
# user_secrets = UserSecretsClient()
# wandb_api = user_secrets.get_secret("wandb_api")

# # Wandb
# wandb.init(entity='upscale-dudes', project='csc-hackathon-2023')
# wandb_logger = WandbLogger(entity='upscale-dudes', project="csc-hackathon-2023")

# Dataset paths
# Train
train_hr_path = '/kaggle/input/fairface-lq-10/train/256_256'
train_lr_path = '/kaggle/input/fairface-lq-10/train/32_32'
# Validation
val_hr_path = '/kaggle/input/fairface-lq-10/validation/256_256'
val_lr_path = '/kaggle/input/fairface-lq-10/validation/32_32'
# Test
test_hr_path = '/kaggle/input/fairface-lq-10/test/256_256'
test_lr_path = '/kaggle/input/fairface-lq-10/test/32_32'

BATCH_SIZE = 256

#### Data modules

In [14]:
class UpscalingDataset(Dataset):
    def __init__(self, lr_folder, hr_folder):
        self.hr_folder = hr_folder
        self.lr_folder = lr_folder
        self.hr_images = sorted(os.listdir(hr_folder))
        self.lr_images = sorted(os.listdir(lr_folder))

    def __len__(self):
        return len(self.hr_images)

    def __getitem__(self, index):
        hr_img_name = self.hr_images[index]
        lr_img_name = self.lr_images[index]

        hr_img_path = os.path.join(self.hr_folder, hr_img_name)
        lr_img_path = os.path.join(self.lr_folder, lr_img_name)

        hr_img = Image.open(hr_img_path).convert('RGB')
        lr_img = Image.open(lr_img_path).convert('RGB')
        lr_img = np.array(lr_img, dtype=np.float32)
        hr_img = np.array(hr_img, dtype=np.float32)
        lr_img /= 255.
        hr_img /= 255.
        lr_img = lr_img.transpose([2, 0, 1])
        hr_img = hr_img.transpose([2, 0, 1])

        return torch.tensor(lr_img, dtype=torch.float), torch.tensor(hr_img, dtype=torch.float)


class UpscalingDataModule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            self.upscaling_train = UpscalingDataset(train_lr_path, train_hr_path)
            self.upscaling_val = UpscalingDataset(val_lr_path, val_hr_path)
        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.upscaling_test = UpscalingDataset(test_lr_path, test_hr_path)

    def train_dataloader(self):
        return DataLoader(self.upscaling_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.upscaling_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.upscaling_test, batch_size=self.batch_size)


#### Custom model template

In [15]:
# class UpscalingModel(pl.LightningModule):
#     def __init__(self):
#         super().__init__()
#         # Initialize your model here

#     def forward(self, x):
#         # Forward pass implementation should go here
#         pass
    
#     def calculate_metrics(self, high_res, low_res):
#         mse_loss = F.mse_loss(high_res, low_res)
#         psnr = 10 * torch.log10(1 / mse_loss)
#         ssim_val = ssim(high_res, low_res, data_range=1.0, size_average=True)
#         return {'mse': mse_loss, 'psnr': psnr, 'ssim': ssim_val}

#     def training_step(self, batch, batch_idx):
#         # Training step implementation should go here        
#         # self.log_dict({f'train_{k}': v for k, v in metrics.items()})
#         pass

#     def validation_step(self, batch, batch_idx):
#         # Validation step implementation should go here
#         # self.log_dict({f'val_{k}': v for k, v in metrics.items()})
#         pass

#     def configure_optimizers(self):
#         # Define your optimizer and learning rate scheduler here
#         pass


#### BICUBIC interpolation model

In [16]:
import torch
from torch import nn
from torch.nn import functional as F
from pytorch_lightning import LightningModule, Trainer
from pytorch_msssim import ssim
from torchvision.transforms.functional import to_pil_image

class UpscalingDNN(LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
        )

        self.interp = nn.Upsample((256, 256), mode='bilinear', align_corners=False)
        self.learning_rate = 1e-3

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.interp(x)

    def training_step(self, batch, batch_idx):
        lr_img, hr_img = batch
        out = self(lr_img)
        metrics = self.calculate_metrics(hr_img, out)
        self.log_dict({f'train_{k}': v for k, v in metrics.items()}, on_step=True, on_epoch=True)
        return metrics['mse']
    
    def validation_step(self, batch, batch_idx):
        lr_img, hr_img = batch
        out = self(lr_img)
        metrics = self.calculate_metrics(hr_img, out)
        self.log_dict({f'val_{k}': v for k, v in metrics.items()}, on_step=True, on_epoch=True)
        if self.current_epoch % 2 == 0:  # Log images every 5 epochs
            # Convert tensors to PIL Images
            hr_img = to_pil_image(hr_img[0])
            lr_img = to_pil_image(lr_img[0])
            out = to_pil_image(out[0])
            # Log images to wandb
            self.logger.experiment.log({
                "hr_images": wandb.Image(hr_img),
                "lr_images": wandb.Image(lr_img),
                "out_images": wandb.Image(out),
            })
            
#     def validation_step(self, batch, batch_idx):
#         lr_img, hr_img = batch
#         out = self(lr_img)
#         metrics = self.calculate_metrics(hr_img, out)
#         self.log_dict({f'val_{k}': v for k, v in metrics.items()}, on_step=True, on_epoch=True)

    def calculate_metrics(self, high_res, low_res):
        mse_loss = F.mse_loss(high_res, low_res)
        psnr = 10 * torch.log10(1 / mse_loss)
        ssim_val = ssim(high_res, low_res, data_range=1.0)
        return {'mse': mse_loss, 'psnr': psnr, 'ssim': ssim_val}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)


#### Custom callbacks

In [17]:
from pytorch_lightning.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_mse',  # Specify the validation loss to monitor
    dirpath='/kaggle/working/models/',  # Directory where the models will be saved
    filename='upscaling-{epoch:02d}-{val_mse:.3f}-{val_psnr:.3f}',  # Template for the saved model's name
    save_top_k=1,  # Save only the best model
    mode='min',  # Minimize validation loss
)


#### Train

In [18]:
# Wandb
wandb.init(entity='upscale-dudes', project='csc-hackathon-2023')
wandb_logger = WandbLogger(entity='upscale-dudes', project="csc-hackathon-2023")

# Prepare data
data_module = UpscalingDataModule(BATCH_SIZE)

# Define your model
model = UpscalingDNN()

# Fit the model
trainer = pl.Trainer(max_epochs=5, callbacks=[checkpoint_callback], logger=wandb_logger)  # use wandb_logger for Weights & Biases logging
trainer.fit(model, data_module)

epoch,▁▁▃▃▃▅▅▅▁▁▃▃▃▅▅▅▆▆███
train_mse_epoch,█▂▂▆▂▂▁▁
train_mse_step,█▃█▃▁
train_psnr_epoch,▁▅▆▂▅▆▇█
train_psnr_step,▁▅▁▆█
train_ssim_epoch,▁▆▇▃▆▇▇█
train_ssim_step,▁▅▁▄█
trainer/global_step,▁▁▁▂▂▃▁▁▄▄▅▁▁▁▅▅▁▁▂▂▃▁▁▁▄▅▁▁▁▅▅▁▁▇▇▇▁▂▂█
val_mse_epoch,█▅▃█▅▂▁▁
val_mse_step,██▆▅▅▄▃▄▃▇█▆▅▅▄▃▃▂▂▂▁▂▂▁
val_psnr_epoch,▁▃▅▁▃▆▇█


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#### Train template

In [19]:
# # Prepare data
# data_module = UpscalingDataModule(BATCH_SIZE)

# # Define your model
# model = UpscalingModel()

# # Fit the model
# trainer = pl.Trainer(max_epochs=10, logger=wandb_logger)
# trainer.fit(model, data_module)
